In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Google API Key
from config import gkey

ModuleNotFoundError: No module named 'config'

In [34]:
# Import previously created cities_1 data
cities_df = pd.read_csv('../output_data/cities_1.csv',index_col=False)
cities_df.drop(columns=["Unnamed: 0"], inplace=True)
cities_df.head()

,City,Country_Code,Input_Lat,Input_Lon,Actual_Lat,Actual_Lon,MaxTemp,Humidity,Cloudiness,Wind_Speed,Date_Time,Hemisphere
0,hobart,AU,-73,138,-42.8794,147.3294,55.40,71,98,9.22,5/1/2021 - 9:16:29,Southern
1,albany,US,-78,98,42.6001,-73.9662,41.00,70,23,1.01,5/1/2021 - 9:16:30,Northern
2,esperance,AU,-29,124,-33.8667,121.9000,55.40,88,5,5.75,5/1/2021 - 9:18:26,Southern
3,dikson,RU,72,75,73.5069,80.5464,24.85,88,94,7.83,5/1/2021 - 9:17:38,Northern
4,hilo,US,34,-143,19.7297,-155.0900,69.80,73,40,4.61,5/1/2021 - 9:17:0,Northern


In [35]:
# Store latitude and longitude in locations
locations = cities_df[["Actual_Lat", "Actual_Lon"]]


# store humidities in humidity
humidity = cities_df["Humidity"].astype(float)

In [36]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=1)

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [37]:
# Ideal conditions are temperature between 70 and 80 F, Wind_speed < 15 mph and humidity < 50%
cities_df[["Hotel_Name","Hotel_Address","Ideal"]]= " "
ideal_df = cities_df.loc[lambda x: (x.Humidity < 50) & (x.MaxTemp > 70) & (x.MaxTemp < 80) & (x.Wind_Speed < 15)][["City","Actual_Lat","Actual_Lon"]]
ideal_df.count()

City          11
Actual_Lat    11
Actual_Lon    11
dtype: int64

In [38]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_df.iterrows():
    # get lat, lng from df
    lat = row["Actual_Lat"]
    lng = row["Actual_Lon"]
    city = row["City"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

        # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:

        print(f"looking for hotel near {city} ",end = "")

        cities_df.loc[index,"Hotel_Name"] = name_address["results"][0]["name"]
        cities_df.loc[index,"Hotel_Address"] = name_address["results"][0]["vicinity"]
        cities_df.loc[index,"Ideal"] = True

        print("Success")
        hotel_name = name_address["results"][0]["name"]
        print(f"Found {hotel_name}!!!")

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    # if index > 5:
        # break

looking for hotel near hermanus Success
Found Harbour House Hotel!!!
looking for hotel near viedma Success
Found Hotel Austral Viedma!!!
looking for hotel near mahdalynivka Missing field/result... skipping.
looking for hotel near kuryk Success
Found Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy Kliyentov .!!!
looking for hotel near luderitz Success
Found LÜDERITZ NEST HOTEL!!!
looking for hotel near saldanha Success
Found Saldanha Bay Hotel!!!
looking for hotel near nemyriv Missing field/result... skipping.
looking for hotel near beyneu Missing field/result... skipping.
looking for hotel near makat Missing field/result... skipping.
looking for hotel near tazmalt Success
Found Hotel Rabah Syphax!!!
looking for hotel near damietta Success
Found فندق المنشي دمياط!!!


In [40]:
# Visualize to confirm hotel data appears
cities_df.loc[lambda x: x.Ideal == True].head()

,City,Country_Code,Input_Lat,Input_Lon,Actual_Lat,Actual_Lon,MaxTemp,Humidity,Cloudiness,Wind_Speed,Date_Time,Hemisphere,Hotel_Name,Hotel_Address,Ideal
30,hermanus,ZA,-74,13,-34.4187,19.2345,73.99,44,4,3.00,5/1/2021 - 9:16:33,Southern,Harbour House Hotel,"22 Harbour Rd, Hermanus",True
55,viedma,AR,-44,-60,-40.8135,-62.9967,70.63,34,95,10.87,5/1/2021 - 9:17:21,Southern,Hotel Austral Viedma,"Villarino 292, Costanera del Rio Negro, Viedma",True
81,kuryk,KZ,42,51,43.2000,51.6500,72.32,41,100,6.67,5/1/2021 - 9:18:42,Northern,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...,"ул. Досан Батыра, 20, Kuryk",True
129,luderitz,NaN,-26,10,-26.6481,15.1594,77.22,36,0,10.40,5/1/2021 - 9:19:17,Southern,LÜDERITZ NEST HOTEL,"820 Dias St, Luderitz",True
330,saldanha,ZA,-34,10,-33.0117,17.9442,75.20,31,5,6.91,5/1/2021 - 9:17:1,Southern,Saldanha Bay Hotel,"51 Main Rd, Saldanha",True


In [41]:
# Create a map using state centroid coordinates to set markers
marker_locations = cities_df.loc[lambda x: x.Ideal == True][["Actual_Lat", "Actual_Lon"]]
hotel_name = cities_df.loc[lambda x: x.Ideal == True]["Hotel_Name"]
hotel_address = cities_df.loc[lambda x: x.Ideal == True]["Hotel_Address"]

markers = gmaps.marker_layer(marker_locations, info_box_content=[f"Hotel: {hotel_name[index]}, address: {hotel_address[index]}" for index,row in cities_df.loc[lambda x: x.Ideal == True].iterrows()])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))